### Fix doc_refs in db for PropertyApp export

This notebook fixes missing doc_ref columns in property_transaction table so that Ivana can cross ref the property transaction with the bank record.

It iterates property_transaction records with missing doc_refs, matching up the amount, property code and date with a record from the full bank data for the year.

Requires: 
1. access to Property App VM database, 
2. input spreadsheet with 6045 and 3072 data (see transactions_gen.xls from the previous year)

In [1]:
import numpy as np
import pandas as pd

In [2]:
#path = 'M:\My Documents\Business\Bugisiw Ltd\TaxReturn\TaxReturn_2021-2022\LLP accounts'
taxreturn_yr='TaxReturn_2021-2022'
path = 'E:\\dtuklaptop\\e\\Users\\Mat\\python\\data\\property\\'+taxreturn_yr+'\\LLP accounts\\'
startdatestr='2021-04-06'
enddatestr='2022-04-05'

In [3]:
bank_input_file = path + '/transactions_gen.xlsx'
df6=pd.read_excel(bank_input_file,sheet_name='6045')
df6=df6[~df6.Account.isnull()]
df6=df6.astype({"Memo": str})
df3=pd.read_excel(bank_input_file,sheet_name='3072')
df3=df3[~df3.Account.isnull()]
df3=df3.astype({"Memo": str})
df6['Found']=0
df3['Found']=0

In [4]:
propertyidmap = {
2:'321LON',
1:'F1321LON',
14:'F2321LON',
3:'F3321LON',
4:'F4321LON',
7:'169FAW',
5:'F1169FAW',
6:'F2169FAW',
8:'F3169FAW',
73:'F1171FAW',
74:'F2171FAW',
75:'F3171FAW',
12:'163FRA',
9:'F2163FRA',
10:'F3163FRA',
11:'F4163FRA',
17:'SHOP196KIN',
15:'196AKIN',
44:'196AKIN',
16:'196BKIN',
19:'23BHAM',
34:'23CHAM',
18:'23CHAM',
31:'F58ALH',
30:'F68ALH',
33:'F78ALH',
32:'F88ALH',
26:'F1746ALH',
22:'F1846ALH',
21:'F1946ALH',
25:'F2046ALH',
28:'F2146ALH',
23:'F2246ALH',
27:'F2346ALH',
46:'F31214ALH',
52:'F41214ALH',
47:'F51214ALH',
48:'F71214ALH',
49:'F111214ALH',
53:'F101214ALH',
54:'F141214ALH',
55:'F161214ALH',
66:'F11618ALH',
67:'F31618ALH',
63:'F61618ALH',
68:'F71618ALH',
64:'F111618ALH',
71:'F121618ALH',
70:'F131618ALH',
65:'F161618ALH'
}

In [5]:
import sqlalchemy as sqla
import pandas as pd
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"E:\dtuklaptop\e\Users\Mat\instantclient_19_9")

# Test to see if it will print the version of sqlalchemy
print(sqla.__version__)    # this returns 1.2.15 for me

# Test to see if the cx_Oracle is recognized
print(cx_Oracle.version)   # this returns 8.0.1 for me

# This fails for me at this point but will succeed after the solution described below
cx_Oracle.clientversion()

connection = cx_Oracle.connect(
    user="property_test",
    password="property_test",
    dsn="192.168.2.134/ORCL")

print("Successfully connected to Oracle Database")

1.3.10
8.3.0
Successfully connected to Oracle Database


Iterate property_transaction records, finding corresponding bank transactions to update doc_ref column.

In [12]:
cursor_query = connection.cursor()
cursor_update = connection.cursor()

query=\
'select pty.property_id, pty.alias alias, ptt.trans_id trans_id, ptt.description description, ptt.trans_date trans_date, ptt.amount amount '\
' from property_transaction ptt' \
' ,    property pty' \
' where ptt.property_id = pty.property_id' \
' and ptt.doc_ref is null'\
' and ptt.subcategory_id in (72)'\
' and ptt.trans_date >= TO_DATE(\'26/06/2019\',\'DD/MM/YYYY\')'\
' order by ptt.trans_date asc'

update = 'update property_transaction '\
         'set doc_ref = :doc_ref '\
         'where trans_id = :trans_id'

def reconcile_transactions_with_bank(df_bank, label):
    multiple_found=[]
    not_found=[]
    # For each property_transaction record, find matching record in bank download spreadsheet and update doc_ref in property_transaction
    for property_id, alias, trans_id, desc, trans_date, amount in cursor_query.execute(query):
        #print(trans_date)
        print(trans_id)
        #print(desc)
        #print(str(trans_date) + ' ' + str(trans_id) + ': ' +desc)
        p_id=propertyidmap[property_id]
        rec=df_bank.loc[(df_bank['Property']==p_id)&(df_bank['Amount']==-amount)&(df_bank['Date']==trans_date)]
        if not rec.empty:
            if(rec.shape[0]==1):
                #print(rec.iloc[0].Memo)
                doc_ref=str(label) +': ' + rec.iloc[0].Memo.replace('nan','')
                cursor_update.execute(update, [doc_ref,trans_id])
                df_bank.at[rec.iloc[0].name,'Found']=1
            else:
                cursor_update.execute(update, ['MULTIPLE',trans_id])
                multiple_found.append(trans_id)
        else:
            not_found.append(trans_id)

    #print(multiple_found)
    #print(not_found)
    return multiple_found, not_found 

#connection.commit()
#connection.rollback()

In [ ]:
' and ptt.subcategory_id not in (89,72)'\

In [13]:
reconcile_transactions_with_bank(df6,'6045')

211922
211961
211959
212649
212702
212704
213128
212986
213169
213006
213171
213867
213732
213760
213758
213910
213908
213803
213797
213916
213801
213799
213914
214302
214300
214321
215277
215429
215427
215425
215292
215290
215281
215279
215471
218313
218196
218194
218309
218311
218198
218200
218202
218331
218698
218700
218702
218541
218543
218547
218545
218549
218720
218986
219199
218988
218990
218984
219197
219201
218982
219239
220382
220384
220386
220388
220390
220396
220588
220590
220592
220633
222563
222565
222757
222569
222753
222755
222561
222567
222800
224207
224209
224211
224213
224215
224415
224411
224413
224421
224460
227551
227377
227379
227375
227373
227371
227553
227555
227591
227800
227790
227798
227792
227988
227990
227796
227986
227794
228051
228053
228033
230906
230908
230914
230916
230918
231070
230904
231066
231068
231117
231115
230988
231310
231540
231542
232125
232348
232350
240183
240338
240340
241410
241320


([],
 [211922,
  211961,
  211959,
  212649,
  212702,
  212704,
  213128,
  213169,
  213171,
  213867,
  213910,
  213908,
  213916,
  213914,
  214302,
  214300,
  214321,
  215429,
  215427,
  215425,
  215471,
  218313,
  218309,
  218311,
  218331,
  218698,
  218700,
  218702,
  218720,
  219199,
  219197,
  219201,
  219239,
  220588,
  220590,
  220592,
  220633,
  222757,
  222753,
  222755,
  222800,
  224415,
  224411,
  224413,
  224421,
  224460,
  227551,
  227553,
  227555,
  227591,
  227988,
  227990,
  227986,
  228051,
  228053,
  228033,
  231070,
  231066,
  231068,
  231117,
  231115,
  231540,
  231542,
  232348,
  232350,
  241410])

In [14]:
reconcile_transactions_with_bank(df3,'3072')

211922
211961
211959
212649
212704
212702
213128
213171
213169
213867
213908
213910
213914
213916
214302
214321
214300
215429
215425
215427
215471
218309
218311
218313
218331
218702
218700
218698
218720
219197
219199
219201
219239
220588
220590
220592
220633
222753
222755
222757
222800
224411
224413
224415
224421
224460
227555
227551
227553
227591
227990
227988
227986
228033
228051
228053
231068
231070
231066
231115
231117
231540
231542
232348
232350
241410


([], [])

In [ ]:
connection.rollback()

In [10]:
bank_output_file = path + '/transactions_gen_output.xlsx'
writer = pd.ExcelWriter(bank_output_file, engine='xlsxwriter',datetime_format='dd/mm/yyyy')
df6.to_excel(writer,sheet_name='6045', index=False)
df3.to_excel(writer,sheet_name='3072', index=False)
writer.save()

In [15]:
connection.commit()